## 小車2號：用360度servo

In [33]:
%websocketconnect --password 1234 ws://192.168.43.73:8266

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


In [2]:
%serialconnect to --port=com7 --baud=115200


Closing websocket <websocket._core.WebSocket object at 0x044FD130>
Connecting to --port=com7 --baud=115200 

Try one of these ports as --port= 
  COM18
  COM19

In [34]:
# 定義
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM


In [35]:
ang0=40 #逆時針
ang90=77 #停止
ang180=115 #順時針

In [36]:
p2 = Pin(D4)

In [37]:
p2.value(0)
print(p2.value())

0


### 定義小車輪子


In [38]:
servoL = PWM(Pin(D6), freq=50)
servoR = PWM(Pin(D5), freq=50)

In [39]:
ang0=40 #逆時針
ang90=77 #停止
ang180=115 #順時針

- 左輪

In [25]:
servoL.duty(ang0) # FW

In [26]:
servoL.duty(ang180) #BK

In [27]:
servoL.duty(ang90)

- 左輪

In [22]:
servoR.duty(ang180) #FW

In [23]:
servoR.duty(ang0) #BK

In [24]:
servoR.duty(ang90)

- 前進

In [28]:
servoL.duty(ang0)
servoR.duty(ang180)

- 後退

In [29]:
servoL.duty(ang180)
servoR.duty(ang0)

- 停止

In [40]:
servoL.duty(ang90)
servoR.duty(ang90)

### 超音波功能測試

- ping

In [41]:
def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58

In [47]:
print(ping(trigPin=D7,echoPin=D8))

9.27586


In [48]:
while True:
    print('%s cm' %(ping(trigPin=D7,echoPin=D8)))
    time.sleep(1)

4.48276 cm
9.06897 cm
6.39655 cm
6.39655 cm
9.05172 cm
9.84483 cm
.7.58621 cm
13.7931 cm
18.3448 cm
6.37931 cm
6.39655 cm
.6.39655 cm
6.39655 cm
6.39655 cm
6.39655 cm


*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
KeyboardInterrupt: 


### car2 第一版程式

In [185]:
%websocketconnect --password 1234 ws://192.168.43.73:8266

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


In [186]:
%sendtofile main.py

# 定義
import machine
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

ang0=40 #逆時針
ang90=77 #停止
ang180=115 #順時針

#Setup PINS
servoL = PWM(Pin(D6), freq=50)
servoR = PWM(Pin(D5), freq=50)

def car_fwd():
    servoL.duty(ang180)
    servoR.duty(ang0)

def car_rev():
    servoL.duty(ang0)
    servoR.duty(ang180)

def car_stop():
    servoL.duty(ang90)
    servoR.duty(ang90)

def car_right():
    servoL.duty(ang90)
    servoR.duty(ang0)

def car_left():
    servoL.duty(ang0)
    servoR.duty(ang90)


def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58

# go~
car_fwd()

while(True):
    distance=ping(trigPin=D7,echoPin=D8)
    if distance>20:
        car_fwd()
    elif distance>10:
        car_rev()
        time.sleep(0.5)
        car_right()
        time.sleep(1)
    elif distance<=10:
        car_rev()
        time.sleep(1)
    time.sleep(0.1)

Sent 98 lines (1928 bytes) to main.py.


## car2 程式第二版 2018/07/15

In [1]:
%websocketconnect --password 1234 ws://192.168.43.73:8266

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


### 基本動作測試

In [2]:
# 定義
import machine
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

ang0=40 #逆時針
ang90=77 #停止
ang180=115 #順時針

#Setup PINS
servoL = PWM(Pin(D5), freq=50)
servoR = PWM(Pin(D6), freq=50)

def car_fwd():
    servoL.duty(ang0)    #左前轉
    servoR.duty(ang180)  #右前轉

def car_rev():
    servoL.duty(ang180)  #左後轉
    servoR.duty(ang0)    #右後轉

def car_stop():
    servoL.duty(ang90)
    servoR.duty(ang90)

def car_right():
    servoL.duty(ang0)
    servoR.duty(ang90)

def car_left():
    servoL.duty(ang90)
    servoR.duty(ang0)
    
def car_right2():
    servoL.duty(ang0)
    servoR.duty(ang0)

def car_left2():
    servoL.duty(ang180)
    servoR.duty(ang180)    

In [3]:
car_stop()

In [14]:
car_right()

In [15]:
car_right2()

In [6]:
ang0=40 #逆時針
ang90=77 #停止
ang180=115 #順時針
for i in range(ang0, ang90, 3):
    servoL.duty(i)
    time.sleep(1)
    print([i,],)

.[40]
[43]
[46]
[49]
[52]
[55]
[58]
.[61]
[64]
[67]
[70]
[73]
[76]


In [10]:
# 73變成慢轉
servoL.duty(73)

In [18]:
servoL.duty(40)

In [19]:
%sendtofile main.py

# 定義
import machine
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

ang0=40 #逆時針
ang90=77 #停止
ang180=115 #順時針

#Setup PINS
servoL = PWM(Pin(D5), freq=50)
servoR = PWM(Pin(D6), freq=50)

def car_fwd():
    servoL.duty(ang0)    #左前轉
    servoR.duty(ang180)  #右前轉

def car_rev():
    servoL.duty(ang180)  #左後轉
    servoR.duty(ang0)    #右後轉

def car_stop():
    servoL.duty(ang90)
    servoR.duty(ang90)

def car_right():
    servoL.duty(ang0)
    servoR.duty(ang90)

def car_left():
    servoL.duty(ang90)
    servoR.duty(ang0)
    
def car_right2():
    servoL.duty(ang0)
    servoR.duty(ang0)

def car_left2():
    servoL.duty(ang180)
    servoR.duty(ang180)    

def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58

# go~
car_fwd()

last_distance = 0
while(True):
    distance=ping(trigPin=D7,echoPin=D8)
    
    #if distance == 999:
    #    if last_distance < 10:
    #        car_rev()
    #        time.sleep(1)
            
    if distance>15:
        car_fwd()
    elif distance>10:
        car_right2()
        time.sleep(0.5)
    elif distance<=5:
        car_rev()
        time.sleep(0.5)
        
    last_distance = distance
    time.sleep(0.1)

Sent 112 lines (2235 bytes) to main.py.


In [20]:
%rebootdevice

normal repl mode not detected 
MicroPython v1.9.1-8-g7213e78d on 2017-06-12; ESP module with ESP8266
Type "help()" for more information.
>>> 
>>> 
PYB: soft reboot

not command mode
[Timed out waiting for recognizable response]


In [27]:
%websocketconnect --password 1234 ws://192.168.43.73:8266

** WebSocket connected **
Password: Ready.


In [ ]:
#%sendtofile main.py

# 定義
import machine
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

ang0=40 #逆時針
ang90=77 #停止
ang180=115 #順時針

#Setup PINS
servoL = PWM(Pin(D5), freq=50)
servoR = PWM(Pin(D6), freq=50)

def car_fwd():
    servoL.duty(ang0)    #左前轉
    servoR.duty(ang180)  #右前轉

def car_rev():
    servoL.duty(ang180)  #左後轉
    servoR.duty(ang0)    #右後轉

def car_stop():
    servoL.duty(ang90)
    servoR.duty(ang90)

def car_right():
    servoL.duty(ang0)
    servoR.duty(ang90)

def car_left():
    servoL.duty(ang90)
    servoR.duty(ang0)
    
def car_right2():
    servoL.duty(ang0)
    servoR.duty(ang0)

def car_left2():
    servoL.duty(ang180)
    servoR.duty(ang180)    

def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58

# go~
car_fwd()

last_distance = 0
while(True):
    distance=ping(trigPin=D7,echoPin=D8)
    print("d=%d (%d)" %(distance, last_distance))
    '''
    if distance == 999:
        if last_distance < 10:
            car_rev()
            time.sleep(1)
    '''        
    if distance>15:
        car_fwd()
    elif distance>10:
        car_right2()
        time.sleep(0.5)
    elif distance<=5:
        car_rev()
        time.sleep(0.5)
        
    last_distance = distance
    time.sleep(0.2)